In [ ]:
# twscrape add_accounts <file_path> <line_format>
# line_format should have "username", "password", "email", "email_password" tokens
# tokens delimeter should be same as an file
twscrape add_accounts accounts.txt username:password:email:email_password
twscrape login_accounts


In [ ]:
pip install twscrape
pip instal selenium
pip install bs4

In [ ]:
"User"
id: int
    id_str: str
    url: str
    username: str
    displayname: str
    rawDescription: str
    created: datetime
    followersCount: int
    friendsCount: int
    statusesCount: int
    favouritesCount: int
    listedCount: int
    mediaCount: int
    location: str
    profileImageUrl: str
    profileBannerUrl: str | None = None
    protected: bool | None = None
    verified: bool | None = None
    blue: bool | None = None
    blueType: str | None = None
    descriptionLinks: list[TextLink] = field(default_factory=list)
    _type: str = "snscrape.modules.twitter.User"

"Tweets"
id: int
    id_str: str
    url: str
    date: datetime
    user: User
    lang: str
    rawContent: str
    replyCount: int
    retweetCount: int
    likeCount: int
    quoteCount: int
    conversationId: int
    hashtags: list[str]
    cashtags: list[str]
    mentionedUsers: list[UserRef]
    links: list[TextLink]
    viewCount: int | None = None
    retweetedTweet: Optional["Tweet"] = None
    quotedTweet: Optional["Tweet"] = None
    place: Optional[Place] = None
    coordinates: Optional[Coordinates] = None
    inReplyToTweetId: int | None = None
    inReplyToUser: UserRef | None = None
    source: str | None = None
    sourceUrl: str | None = None
    sourceLabel: str | None = None
    media: Optional["Media"] = None
    _type: str = "snscrape.modules.twitter.Tweet"

In [ ]:
#RUNNING IN JUPYTER NOTEBOOK
#crawl data url from search X key

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json

#modify
PATH = 'F:\slide\chromedriver-win64\chromedriver.exe'
username = "CoiNguyn443321*"
password = "123456coi"

# List of target URLs : tự cho
urls = ['https://twitter.com/search?q=web3&src=typed_query&f=user','https://twitter.com/search?q=blockchain&src=typed_query&f=user','https://twitter.com/search?q=DeFi&src=typed_query&f=user'
,'https://twitter.com/search?q=trend&src=typed_query&f=user', 'https://twitter.com/search?q=crypto%20musk&src=typeahead_click&f=user','https://twitter.com/search?q=Bitcoin&src=typeahead_click&f=user','https://twitter.com/search?q=dollar&src=typeahead_click&f=user','https://twitter.com/search?q=ukraine&src=typeahead_click&f=user']
driver = webdriver.Chrome(PATH)

# Login to Twitter
driver.get("https://twitter.com/login")
time.sleep(15)

result_list = []
for target_url in urls:

    driver.get(target_url)
    time.sleep(10)
    scroll_down_count = 5  # Change this to the number of times you want to scroll down
    for _ in range(scroll_down_count):
    # Scroll down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for a short time to let the page load more content
        time.sleep(7)
        resp = driver.page_source
        soup = BeautifulSoup(resp, 'html.parser')
    #find username
        for user in soup.find_all("div", {"data-testid":"cellInnerDiv"}):
            try:
                username = user.find("div", {"class": "css-1dbjc4n r-1adg3ll r-1ny4l3l"}).find("a", {"class": "css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1wbh5a2 r-dnmrzs r-1ny4l3l"}).text
                userurl = user.find("div", {"class": "css-1dbjc4n r-1adg3ll r-1ny4l3l"}).find("div", {"class": "css-901oao css-1hf3ou5 r-1bwzh9t r-18u37iz r-37j5jr r-1wvb978 r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-qvutc0"}).text
            except:
                pass
            result_list.append({ "username": username, "useurl": userurl})

    
#     try:
#         o['username'] = soup.find("div", {"class": "css-1dbjc4n r-1adg3ll r-1ny4l3l"}).find("div", {"class": "css-1dbjc4n r-1wbh5a2 r-dnmrzs r-1ny4l3l"}).text
#     except:
#         o["profile_name"] = None
    
#     result_list.append(o)
#   print(result_list)
# Save data to JSON file
   
filename = "twitter_profile_url10.json"
with open(filename, 'a', encoding='utf-8') as jsonfile:
    json.dump(result_list, jsonfile, ensure_ascii=False, indent=4)



print(f"Data saved to {filename}")
driver.quit()

In [ ]:
import pandas as pd
df = pd.read_json("twitter_profile_url10.json", encoding="utf-8")

# Loại bỏ các mục trùng lặp dựa trên cột "useurl"
filenew = df.drop_duplicates(subset="useurl", keep="first")

# Ghi DataFrame mới vào một tệp JSON
filenew.to_json("profile_url.json", orient="records", force_ascii=False, indent=4)

In [14]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import json
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        elif hasattr(o, '__dict__'):
            return o.__dict__
        return super().default(o)

async def get_user_additional_info(api, user_id):
    try:
        user = await api.user_by_id(user_id)

        if user is not None:
            user_details = {
                'id': user.id,
                'username': user.username,
                'displayName': user.displayname,
                'url': user.url,
                'intro': user.rawDescription,
                'join_date': user.created,
                'location': user.location,
                'friends_count': user.friendsCount,
                'favorite_count':user.favouritesCount,
                'followers_count': user.followersCount,
                'media_count': user.mediaCount,
                'status_count': user.statusesCount,
                'listed_count': user.listedCount,
                'is_protected': user.protected,
                'is_verified': user.verified,
                'ref': user. descriptionLinks,
            }
            return user_details
        else:
            return None

    except Exception as e:
        print(f"Error fetching user details: {e}")
        return None

async def crawl_user_profile_and_tweets(api, username, limit=20):
    try:
        # Get user information
        user = await api.user_by_login(username)

        if user is not None and hasattr(user, 'id'):
            # Get user tweets
            tweets = await gather(api.user_tweets(user.id, limit=limit))

            # Create a list of dictionaries with user, tweet, and additional user information
            data = {'user': {}, 'tweets': []}
            user_additional_info = await get_user_additional_info(api, user.id)
            if user_additional_info:
                data['user'] = {
                'id': user.id,
                'username': user.username,
                'displayName': user.displayname,
                'url': user.url,
                'intro': user.rawDescription,
                'join_date': user.created,
                'location': user.location,
                'friends_count': user.friendsCount,
                'favorite_count':user.favouritesCount,
                'followers_count': user.followersCount,
                'media_count': user.mediaCount,
                'status_count': user.statusesCount,
                'listed_count': user.listedCount,
                'is_protected': user.protected,
                'is_verified': user.verified,
                'ref': user. descriptionLinks,
                }

            for tweet in tweets:
                tweet_details = await api.tweet_details(tweet.id)
                data['tweets'].append({
                    'user':tweet.user.username,
                    'tweet_id': tweet.id,
                    'tweet_content': tweet.rawContent,
                    'tweet_created_at': tweet.date,
                    'like_count': tweet.likeCount,
                    'retweet_count': tweet.retweetCount,
                    'comment_count': tweet_details.replyCount,
                    'view_count': tweet.viewCount ,
                    'quote_count': tweet.quoteCount,
                    'hashtags': tweet.hashtags,
                    'place': tweet.place,
                    'mentioned_users': tweet.mentionedUsers
                })
            
            # Export data to a JSON file
            filename = "profile_and_tweets.json"
            with open(filename, 'a', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4, cls=DateTimeEncoder)

            print(f"Data for {username} has been successfully exported to file ")
        else:
            print(f"User {username} not found or has no ID.")
    except Exception as e:
        print(f"Error: {e}") 
                 

async def main():
    api = API()
    #await api.pool.add_account("your_username", "your_password", "your_email@example.com", "your_email_password")
    await api.pool.login_all()

    json_file_path = r'C:\Users\nguye\OneDrive - Hanoi University of Science and Technology\code\bigdata\profile_url.json'

    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as file:
        user_list = json.load(file)
        

    for username in user_list:
    # Call the crawl_user_profile_and_tweets function
        await crawl_user_profile_and_tweets(api, username['useurl'].lstrip('@'), limit=2)

    set_log_level("DEBUG")

    # Other actions you may want to perform using the API

if __name__ == "__main__":
    await main()


Data for web3 has been successfully exported to file 
Data for polkastarter has been successfully exported to file 
Data for SecondLiveReal has been successfully exported to file 
Data for rebaked_dao has been successfully exported to file 
Data for mopindo has been successfully exported to file 
Data for MoonGirlBunny1 has been successfully exported to file 
Data for CAWATOKYO has been successfully exported to file 
Data for cengqi099 has been successfully exported to file 
Data for web3isgreat has been successfully exported to file 
Data for Web_3space has been successfully exported to file 
Data for AirdropAchiever has been successfully exported to file 
Data for okxweb3 has been successfully exported to file 
Data for BuyWeb3 has been successfully exported to file 
Data for taskonxyz has been successfully exported to file 
Data for web3_ready has been successfully exported to file 
Data for Web3Alerts has been successfully exported to file 
Data for IvanOnTech has been successfully